Scrap Lunar cycle data from the site for Atlanta region 
https://www.timeanddate.com/moon/phases/usa/atlanta?


In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests as Req
import numpy as np

In [2]:
############################## Input ##############################
# Country 
Country = "USA"
# Place 
Place = "Atlanta"
# Year
Year  = pd.Series(["2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018"])
#Year  = pd.Series(["1916"])
# Base Url
Lunar_bs_url = "https://www.timeanddate.com/moon/phases/"

In [3]:
def build_url(Country,Place,Year):
#Sample url 
#Lunar_url = "https://www.timeanddate.com/moon/phases/usa/atlanta?year=1982"
    Lunar_ret_df = pd.DataFrame(columns=["Year","Country","Place","Lunar_url"])
    for i in range(len(Year)):
        Lunar_url    = Country + "/" + Place + "?year=" + Year[i]
        Lunar_sd     = [[Year[i],Country,Place,Lunar_url]]
        Lunar_tmp_df = pd.DataFrame(Lunar_sd,columns=["Year","Country","Place","Lunar_url"])
        Lunar_ret_df = Lunar_ret_df.append(Lunar_tmp_df) 
    return Lunar_ret_df

In [4]:
Lunar_url_df = pd.DataFrame()
Lunar_url_df = build_url(Country,Place,Year)

In [5]:
Lunar_url_df.reset_index(drop=True,inplace=True)
Lunar_url_df.head()

,Year,Country,Place,Lunar_url
0,2008,USA,Atlanta,USA/Atlanta?year=2008
1,2009,USA,Atlanta,USA/Atlanta?year=2009
2,2010,USA,Atlanta,USA/Atlanta?year=2010
3,2011,USA,Atlanta,USA/Atlanta?year=2011
4,2012,USA,Atlanta,USA/Atlanta?year=2012


#Start Scrapping

In [6]:
Lunar_cycle_df = pd.DataFrame(columns=["Year","Country","Place","Full moon"])

In [7]:
for i in range(len(Lunar_url_df)):
        Lunar_full_url  = Lunar_bs_url +  Lunar_url_df.Lunar_url[i]
#-------- Open the Url
        headers = {"Accept-Language": "en-US,en;q=0.5"}
        page_html  = Req.get(Lunar_full_url,headers=headers)
#--------  Parse the Html      
        page_soup = soup(page_html.content,"html.parser")
        page_table = page_soup.find_all("table",{"class":"tb-sm zebra fw tb-hover"})
        page_table = soup.prettify(page_table[0])
        page_soup  = soup(page_table,"html.parser")
        table_body = page_soup.find_all("tbody")
        table_body = soup.prettify(table_body[0])
        page_soup  = soup(table_body,"html.parser")
        table_month = page_soup.find_all("tr")
        for j in range(len(table_month)):
            table_row   = soup.prettify(table_month[j])
            page_soup   = soup(table_row,"html.parser")
            table_val   = page_soup.findAll(name="td")
            lunar_date  = soup.prettify(table_val[4])
            lunar_date  = lunar_date.rsplit(sep=">")
            Full_moon   = lunar_date[1].rsplit(sep="<")
            Full_moon   = Full_moon[0]
            Full_moon   = Full_moon.replace("\n"," ")
            if Full_moon != " ":
                Full_moon   = Full_moon + " " + Lunar_url_df.Year[i]
                Lunar_cycle_s = [[Lunar_url_df.Year[i],Lunar_url_df.Country[i],Lunar_url_df.Place[i],Full_moon]]
                Lunar_cyc_df  = pd.DataFrame(Lunar_cycle_s,columns=["Year","Country","Place","Full moon"])
                Lunar_cyc_df["Full moon"] = Lunar_cyc_df["Full moon"].str.strip()
                Lunar_cycle_df = Lunar_cycle_df.append(Lunar_cyc_df)
#        Cont_1
        

In [8]:
Lunar_cycle_df.reset_index(drop=True,inplace=True)
Lunar_cycle_df.head()

,Year,Country,Place,Full moon
0,2008,USA,Atlanta,21 Jan 2008
1,2008,USA,Atlanta,19 Feb 2008
2,2008,USA,Atlanta,20 Mar 2008
3,2008,USA,Atlanta,19 Apr 2008
4,2008,USA,Atlanta,18 May 2008


In [9]:
#Write to File 
Lunar_cycle_df.to_csv("Lunar_cycle.csv")